# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download('punkt', 'wordnet', 'stopwords')
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multioutput import MultiOutputClassifier
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [2]:
# load data from database
engine = create_engine('sqlite:///message.db')
df = pd.read_sql_table('message', engine)
X = df['message']
Y = df.iloc[:, 4:]

In [3]:
X.shape

(3000,)

In [4]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [ ]:
#Y.sum() / Y.shape[0]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    The function normalizes the string, removes the punctuation characters, tokenizes and lemmatizes the string
    Input: string
    Output: clean tokens
    """
    lemmatizer = WordNetLemmatizer()
    # Case Normalization
    text = text.lower()
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = word_tokenize(text)
    clean_tokens = []
    for tok in text:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    # remove stop words
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# train classifier
pipeline.fit(X_train, y_train)

CPU times: user 9.65 s, sys: 1.36 s, total: 11 s
Wall time: 11.1 s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def evaluate_model(model, X_test, y_test, category):
    """
    input: estimator(model), testing set, testing labels, the category for the lables
    output: 
    """
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, target_names=category, digits=3))

In [8]:
%%time
evaluate_model(pipeline, X_test, y_test, y_test.columns.tolist())

                        precision    recall  f1-score   support

               related      0.864     0.931     0.896       608
               request      0.834     0.783     0.808       411
                 offer      0.000     0.000     0.000         3
           aid_related      0.795     0.780     0.788       423
          medical_help      0.600     0.081     0.143        74
      medical_products      0.750     0.176     0.286        34
     search_and_rescue      0.000     0.000     0.000        34
              security      0.000     0.000     0.000        13
              military      0.000     0.000     0.000         4
                 water      0.929     0.782     0.849       133
                  food      0.897     0.682     0.775       192
               shelter      0.800     0.346     0.483       104
              clothing      0.000     0.000     0.000         7
                 money      0.000     0.000     0.000        13
        missing_people      0.000     0

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
%%time
parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
         'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [100, 200]
        #'clf__min_samples_split': [2, 3 , 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

CPU times: user 6min 16s, sys: 30.3 s, total: 6min 46s
Wall time: 6min 52s


In [10]:
cv.best_params_

{'clf__estimator__n_estimators': 200, 'tfidf__use_idf': False}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
%%time
evaluate_model(cv, X_test, y_test, y_test.columns.tolist())

                        precision    recall  f1-score   support

               related      0.853     0.961     0.903       608
               request      0.832     0.830     0.831       411
                 offer      0.000     0.000     0.000         3
           aid_related      0.824     0.830     0.827       423
          medical_help      0.778     0.095     0.169        74
      medical_products      0.857     0.176     0.293        34
     search_and_rescue      0.000     0.000     0.000        34
              security      0.000     0.000     0.000        13
              military      0.000     0.000     0.000         4
                 water      0.930     0.805     0.863       133
                  food      0.929     0.880     0.904       192
               shelter      0.867     0.500     0.634       104
              clothing      0.000     0.000     0.000         7
                 money      0.000     0.000     0.000        13
        missing_people      0.000     0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
%%time
# Use Naive Bayes algorithms
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None)))
])

parameters = {
    'clf__estimator__alpha': [0.05, 0.1],
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False)
        #'clf__estimator__n_estimators': [10, 50, 100] #, 200],
        #'clf__min_samples_split': [2, 3 , 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

CPU times: user 2min 19s, sys: 27.3 s, total: 2min 46s
Wall time: 2min 48s


In [17]:
cv.best_params_

{'clf__estimator__alpha': 0.05, 'tfidf__use_idf': False}

In [18]:
%%time
evaluate_model(cv, X_test, y_test, y_test.columns.tolist())

                        precision    recall  f1-score   support

               related      0.864     0.964     0.911       608
               request      0.773     0.810     0.791       411
                 offer      0.000     0.000     0.000         3
           aid_related      0.765     0.837     0.799       423
          medical_help      0.600     0.041     0.076        74
      medical_products      0.500     0.088     0.150        34
     search_and_rescue      0.000     0.000     0.000        34
              security      0.000     0.000     0.000        13
              military      0.000     0.000     0.000         4
                 water      0.786     0.496     0.608       133
                  food      0.793     0.740     0.765       192
               shelter      0.727     0.231     0.350       104
              clothing      0.000     0.000     0.000         7
                 money      0.000     0.000     0.000        13
        missing_people      0.000     0

In [20]:
%%time
# Use Adaboost
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))
])

parameters = {
    'clf__estimator__learning_rate': [0.5, 1],
#    'clf__estimator__alpha': [0.1, 0.5, 1]
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False)
        #'clf__estimator__n_estimators': [10, 50, 100] #, 200],
        #'clf__min_samples_split': [2, 3 , 4]
    }
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

CPU times: user 3min 31s, sys: 26.8 s, total: 3min 58s
Wall time: 3min 59s


In [21]:
cv.best_params_

{'clf__estimator__learning_rate': 0.5, 'tfidf__use_idf': True}

In [22]:
%%time
evaluate_model(cv, X_test, y_test, y_test.columns.tolist())

                        precision    recall  f1-score   support

               related      0.863     0.956     0.907       608
               request      0.845     0.781     0.812       411
                 offer      0.000     0.000     0.000         3
           aid_related      0.840     0.757     0.796       423
          medical_help      0.714     0.135     0.227        74
      medical_products      0.556     0.294     0.385        34
     search_and_rescue      0.200     0.029     0.051        34
              security      0.400     0.154     0.222        13
              military      0.000     0.000     0.000         4
                 water      0.910     0.910     0.910       133
                  food      0.920     0.896     0.908       192
               shelter      0.780     0.615     0.688       104
              clothing      0.500     0.571     0.533         7
                 money      0.444     0.308     0.364        13
        missing_people      0.000     0

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.